In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from functions import *
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile




2022-11-11 13:59:23.722842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Computational Tool Final Project**


In [2]:
clean_books1 = pd.read_csv('generalized_books_and_ratings.csv')
clean_books1 = clean_books1[["Title","description","authors","publishedDate","categories","averageRating"]]
clean_books1['clean_summary'] = clean_books1['description'].apply(lambda x: clean_books(x))
#clean_books1['clean_authors'] = clean_books1['authors'].apply(lambda x: clean_books(x))
clean_books1['clean_categories'] = clean_books1['categories'].apply(lambda x: clean_books(x))
clean_books1 = clean_books1.dropna()
books = clean_books1[["Title" ,"authors" ,"publishedDate" , "clean_summary" , "clean_categories" , "averageRating"] ]
books
books.to_csv("final.csv")




In this project we are going to use a cleaned up version of this data set from Kaggle (https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=books_data.csv&fbclid=IwAR3H0BlyYyiNodMoF0GbzknzNMyriDa_9eL00g1if_Y29QBPcobfeHQpZCE).

This dataset contains two csv files, one containg the ratings and another containg the books. Post cleanup there are 44285 books with  columns 
,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
,Title,description,authors,publishedDate,categories,ratingsCount
* id 
* book name 
* genre 
* summary

The second csv file contains the reviews for the books and has 1502433 reviews total after cleanup (removing the ratings for the books we removed in the cleanup of the first csv file). The columns it has is:
,Id,Title,Price,review/helpfulness,review/score,review/time,review/summary,review/text
,Id,Title,Price,User_id,review/score

Our main goal is to classify the books with text simmilarity ( using summaries) and predict their genre correctly. Moreover, we want to make a simple recomendation UI using github pages which the user can choose between time, rating , genre and get some recomendation based on our classification model.


In [35]:
file1 = pd.read_csv("final.csv")
file1['clean_summary'] = file1['clean_summary'].apply(lambda x: clean_books(x))


In [51]:

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
for index, row in file1["clean_summary"].iteritems():
    if  isinstance(row, str):
        cat_arr = row.split(" ")
        for word in cat_arr:
            if word in stop_words:
                cat_arr.remove(word)
        file1['clean_summary'].loc[index] = cat_arr

In [52]:
file1

,Unnamed: 0,Title,authors,publishedDate,clean_summary,clean_categories,averageRating
0,0,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],1996,"[church, christ, biblical, ecclesiology, today...",religion,4.50
1,1,Voices from the Farm: Adventures in Community ...,['Rupert Fike'],2012-08-21,"[twenty, five, years, ago, height, counter, cu...",biography autobiography,4.82
2,2,The Battleship Bismarck,['Stefan Draminski'],2018-09-20,"[bismarck, perhaps, famous, notorious, warship...",history,3.50
3,3,Tess and the Highlander,['May Mcgoldrick'],2002-11,"[on, windswept, isle, of, scotland, seventeen,...",fiction,4.24
4,4,Beginner's Yoruba (Hippocrene Beginner's Series),['Kayode J. Fakinlede'],2005,"[, beginners, yoruba, now, available, two, acc...",education,3.80
...,...,...,...,...,...,...,...
41401,41830,Thin Within,"['Judy Halliday', 'Arthur Halliday']",2005-04-17,"[want, lose, weight, dieting, doesnt, seem, wo...",health fitness,4.93
41402,41831,The Awakening and Selected Stories (Modern Lib...,['Kate Chopin'],2003,"[it, first, appeared, awakening, greeted, crie...",love stories,3.00
41403,41832,Final things,['Jenny Offill'],2015-03-17,"[graces, father, believes, science, builds, da...",fiction,4.00
41404,41833,The Orphan Of Ellis Island (Time Travel Advent...,['Elvira Woodruff'],2000-06-01,"[school, trip, ellis, island, dominick, avaro,...",fiction,4.68


In [37]:
file1['clean_summary'] = file1['clean_summary'].apply(lambda x: listToString(x))
file1


,Unnamed: 0,Title,authors,publishedDate,clean_summary,clean_categories,averageRating
0,0,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],1996,in the church of christ a biblical ecclesiolog...,religion,4.50
1,1,Voices from the Farm: Adventures in Community ...,['Rupert Fike'],2012-08-21,twenty five years ago at the height of the cou...,biography autobiography,4.82
2,2,The Battleship Bismarck,['Stefan Draminski'],2018-09-20,the bismarck is perhaps the most famous and no...,history,3.50
3,3,Tess and the Highlander,['May Mcgoldrick'],2002-11,in on a windswept isle off of scotland sevente...,fiction,4.24
4,4,Beginner's Yoruba (Hippocrene Beginner's Series),['Kayode J. Fakinlede'],2005,beginners yoruba is now available with two ac...,education,3.80
...,...,...,...,...,...,...,...
41401,41830,Thin Within,"['Judy Halliday', 'Arthur Halliday']",2005-04-17,i want to lose weight but dieting just doesnt ...,health fitness,4.93
41402,41831,The Awakening and Selected Stories (Modern Lib...,['Kate Chopin'],2003,when it first appeared in the awakening was gr...,love stories,3.00
41403,41832,Final things,['Jenny Offill'],2015-03-17,graces father believes in science and builds h...,fiction,4.00
41404,41833,The Orphan Of Ellis Island (Time Travel Advent...,['Elvira Woodruff'],2000-06-01,during a school trip to ellis island dominick ...,fiction,4.68


In [53]:
test = file1[[ 'clean_summary' , 'Title'] ]


In [42]:
test_words =  tuple(test['clean_summary'])
test_hash = tuple(test['Title'])

In [44]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [59]:
docs = [TaggedDocument(
             words=[word for word in test_words.lower().split()],
             tags=[i]
         ) for i, test_words in enumerate(test_hash)]




    

In [62]:
max_epochs = 40
vec_size = 200
alpha = 0.0025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.0025,
                min_count=0.00001,
                dm =1)
  
model.build_vocab(docs)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(docs,
                total_examples=model.corpus_count,
                epochs= max_epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
Model Saved


In [65]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")


# to find most similar doc using tags
similar_doc = model.docvecs.most_similar(2)
print(similar_doc)
#print(model.docvecs['1'])

[(13123, 0.998285174369812), (22398, 0.9980025887489319), (1324, 0.9979535937309265), (188, 0.9979436993598938), (28364, 0.9979380965232849), (5379, 0.997916042804718), (24235, 0.9979107975959778), (466, 0.997910737991333), (16369, 0.9979103803634644), (21210, 0.9978848695755005)]


/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_28069/3041915706.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(2)


In [70]:
similar_doc = model.dv.most_similar(56)
print(similar_doc)

[(27366, 0.9524111151695251), (11475, 0.9512318968772888), (25420, 0.9501119256019592), (40197, 0.9496240615844727), (37306, 0.9478294253349304), (419, 0.9476994276046753), (40069, 0.9458490610122681), (13039, 0.9453433752059937), (39911, 0.9452923536300659), (3190, 0.9446525573730469)]
